# Giriş işlemleri

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import torch
import pandas as pd
from tqdm.auto import tqdm
from typing import Optional, Tuple

In [ ]:
def is_colab() -> bool:
    """
    Google Colab ortamında çalışıp çalışmadığını kontrol eden fonksiyon.

    Args:
        None

    Returns:
        bool: Eğer kod Google Colab'da çalışıyorsa True, aksi halde False döndürür
    """
    try:
        import google.colab
        return True
    except ImportError:
        return False

In [ ]:
# Kök dizin belirleme
if is_colab():
    """
    Eğer kod Google Colab ortamında çalışıyorsa, Google Drive'ı bağlar ve
    kök dizini Google Drive içindeki "turkish_gpt2_finetuning" klasörü olarak ayarlar.
    """
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive'ı Colab ortamına bağlar
    kok_dizin = "/content/drive/MyDrive/turkish_gpt2_finetuning"  # Drive içindeki çalışma klasörünü belirler
else:
    """
    Eğer kod yerel bir ortamda çalışıyorsa, kök dizini mevcut çalışma dizini olarak ayarlar.
    """
    kok_dizin = os.getcwd()  # Mevcut çalışma dizinini alır

# Belirlenen kök dizini kullanıcıya bilgi olarak gösterir
print(f"Kök dizin: {kok_dizin}\n Not: eğer colab kullanıyorsanız, dizini değiştirmeniz gerekir.")

In [ ]:
veri_kumesi_yolu = os.path.join(kok_dizin, "sinama_sorulari.csv")
sonuc_dizini     = os.path.join(kok_dizin, "sonuclar")
# Model kaydetme dizinleri
# gpt4o verisiyle eğitilmiş modeller
gpt2_medium_kaydetme_dizini_gpt4o = os.path.join(kok_dizin, "gpt2_medium_gpt4o")
gpt2_large_kaydetme_dizini_gpt4o = os.path.join(kok_dizin, "gpt2_large_gpt4o")

# deepseek verisiyle eğitilmiş modeller
gpt2_medium_kaydetme_dizini_deepseek = os.path.join(kok_dizin, "gpt2_medium_deepseek")
gpt2_large_kaydetme_dizini_deepseek = os.path.join(kok_dizin, "gpt2_large_deepseek")

model_yollari = {
    "gpt2_medium_gpt4o"   : gpt2_medium_kaydetme_dizini_gpt4o,
    "gpt2_large_gpt4o"    : gpt2_large_kaydetme_dizini_gpt4o,
    "gpt2_medium_deepseek": gpt2_medium_kaydetme_dizini_deepseek,
    "gpt2_large_deepseek" : gpt2_large_kaydetme_dizini_deepseek,
}

soru_sutunu = "soru"

# Özel token tanımlamaları (sınama biçimi için)
soru_baslangic = "<SORU>"
soru_bitis = "</SORU>"
cevap_baslangic = "<CEVAP>"
cevap_bitis = "</CEVAP>"

# Fonksiyonlar

## Soruyu girdi biçimine çevirme fonksiyonu

In [ ]:
def soruyu_girdi_bicimine_cevir(soru: str) -> str:
    """
    Verilen ham soruyu, dil modeline uygun istem (prompt) biçimine dönüştürür.

    Fonksiyon; dışarıda global olarak tanımlı özel etiketleri (<SORU>, </SORU>, 
    <CEVAP>) kullanarak aşağıdaki kalıpta bir metin hazırlar:

        "<SORU> {soru} </SORU> <CEVAP> "

    Model tarafında generate() çağrısı sırasında, </CEVAP> veya EOS token’ı
    üretildiğinde üretim sonlandırılır ve yalnızca cevaba ait kısım alınır.
    Böylece sorunun başı ve bitişi ile cevabın başlangıcı tutarlı bir şekilde
    modellenmiş olur.

    Args:
        soru (str): Kullanıcıdan gelen ham soru metni. Baş-son boşlukları
                    fonksiyon içinde temizlenir (str.strip()).

    Returns:
        str: Dil modeline beslenmeye hazır, etiketlenmiş prompt dizgesi.

    Örnek:
        >>> soruyu_girdi_bicimine_cevir("Türkiye'nin başkenti neresidir?")
        '<SORU> Türkiye'nin başkenti neresidir? </SORU> <CEVAP> '
    """

    # ---------------------------------------------------------- #
    # 1) Giriş temizliği
    # ---------------------------------------------------------- #
    # Sorunun başında / sonunda fazladan boşluk varsa yok sayılır.
    soru = soru.strip()

    # ---------------------------------------------------------- #
    # 2) Prompt kalıbını oluştur
    # ---------------------------------------------------------- #
    # Global sabitler:
    #   soru_baslangic = "<SORU>"
    #   soru_bitis     = "</SORU>"
    #   cevap_baslangic= "<CEVAP>"
    #
    # Model, </CEVAP> token’ına (veya eos_token_id’ye) ulaştığında duracağı
    # için cevap_bitis etiketine şimdi ihtiyaç yok; generate sonrasında
    # kesilecek.
    prompt = f"{soru_baslangic} {soru} {soru_bitis} {cevap_baslangic} "

    # ---------------------------------------------------------- #
    # 3) Sonucu döndür
    # ---------------------------------------------------------- #
    return prompt


## Modeli yükleme fonksiyonu

In [ ]:
def model_yukle(model_yolu: str, dtype: str = "auto") -> Tuple[AutoModelForCausalLM, AutoTokenizer]:
    """ 
    İstenen yolu kullanarak, bitsandbytes vb. ek paketler olmaksızın 
    AutoModelForCausalLM ve ilgili AutoTokenizer nesnelerini yükler.  
    Cihaz (CPU / GPU) ve veri tipi (dtype) seçimini akıllıca yapar, 
    tokenizer’ın pad token’ını garanti altına alır ve (model, tokenizer) ikilisini döndürür.

    Args:
        model_yolu (str): HuggingFace uyumlu, ön-eğitilmiş veya ince-ayarlı model klasörü / HF hub adı.
        dtype (str, optional): Kullanılacak numerik veri tipi.
            "auto" → GPU var ise (Ampere ve sonrası: bfloat16, öncesi: float16); GPU yoksa float32  
            "fp16" → torch.float16  
            "bf16" → torch.bfloat16  
            "fp32" → torch.float32  
            Varsayılan değer "auto"dur.

    Returns:
        Tuple[AutoModelForCausalLM, AutoTokenizer]:
            0. indeks → AutoModelForCausalLM: Yüklenen dil modeli  
            1. indeks → AutoTokenizer      : Eşleşen tokenizer
    """

    # --------------------------------------------------------------------- #
    # 1) dtype seçimi
    # --------------------------------------------------------------------- #
    # Kullanıcı "auto" talep ettiyse, hangi veri tipini kullanacağımıza
    # donanıma bakarak karar veriyoruz.
    if dtype == "auto":
        if torch.cuda.is_available():                          # GPU mevcut mu?
            major_cc = torch.cuda.get_device_capability(0)[0]  # Compute capability (ör. 8.x ⇒ Ampere)
            # Ampere (>= 8.x) kartlar bfloat16 destekler; daha eskilerde float16 güvenlidir.
            kullanici_dtype = torch.bfloat16 if major_cc >= 8 else torch.float16
        else:
            # GPU yok → güvenli yol: float32
            kullanici_dtype = torch.float32
    # Kullanıcı doğrudan dtype belirtmişse eşle
    elif dtype == "fp16":
        kullanici_dtype = torch.float16
    elif dtype == "bf16":
        kullanici_dtype = torch.bfloat16
    else:  # "fp32" veya bilinmeyen değerler
        kullanici_dtype = torch.float32

    # --------------------------------------------------------------------- #
    # 2) Model ve tokenizer’ı yükle
    # --------------------------------------------------------------------- #
    print(f"\n[MODEL] '{model_yolu}' yükleniyor (dtype={kullanici_dtype})...")

    # device_map="auto" → HF, model katmanlarını otomatik olarak CPU / GPU’ya dağıtır.
    model = AutoModelForCausalLM.from_pretrained(
        model_yolu,
        torch_dtype=kullanici_dtype,
        device_map="auto"
    )

    # use_fast=True → mümkünse Rust tabanlı hızlı tokenizer kullan.
    tokenizer = AutoTokenizer.from_pretrained(model_yolu, use_fast=True)

    # --------------------------------------------------------------------- #
    # 3) Pad token güvenliği
    # --------------------------------------------------------------------- #
    # Bazı GPT-2 türevlerinde pad_token tanımlı değildir.
    # Yoksa, eos_token’ı pad_token olarak atayarak generate() çağrılarını
    # sorunsuz hâle getiriyoruz.
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # --------------------------------------------------------------------- #
    # 4) Model & tokenizer döndür
    # --------------------------------------------------------------------- #
    return model, tokenizer

## Modelden cevabı alma fonksiyonu

In [ ]:
def cevap_uret(
        model: AutoModelForCausalLM,
        tokenizer: AutoTokenizer,
        prompt: str,
        *,
        max_new_tokens: int = 1024,
        no_repeat_ngram_size: int = 3,
        repetition_penalty: float = 1.05,
        eos_token_id: Optional[int] = None
) -> str:
    """
    Dil modelinden (Causal Language Model) deterministik olarak cevap üretir.

    Fonksiyon; verilen prompt’un ( <SORU> … <CEVAP> şeklinde hazırlanmış ) 
    devamında, model.generate ile greedy (do_sample=False) stratejisi kullanarak  
    maksimum `max_new_tokens` uzunluğunda metin oluşturur ve </CEVAP> etiketine
    (veya belirtilen eos_token_id’ye) ulaştığında durur.   
    Dönüşte yalnızca cevabı (etiketsiz hâliyle) verir.

    Args:
        model (AutoModelForCausalLM): Cevap üretilecek, causal LM başlıklı model.
        tokenizer (PreTrainedTokenizerBase): Prompt’u token’layacak/çıktıyı çözümleyecek tokenizer.
        prompt (str): Modelin başlangıç girdi dizisi.
        max_new_tokens (int, optional): Üretilecek maksimum yeni token sayısı. Varsayılan 1024.
        no_repeat_ngram_size (int, optional): ‘n-gram’ tekrarı engelleme boyutu. Varsayılan 3.
        repetition_penalty (float, optional): Tekrarlanan token’lar için ceza katsayısı (>1 arttırır). Varsayılan 1.05.
        eos_token_id (Optional[int], optional): </CEVAP> token’ının id’si. 
            None ise fonksiyon tokenizer ile hesaplar.

    Returns:
        str: Modelin ürettiği cevabın (etiketsiz) son hâli.

    Notlar:
        - Greedy (do_sample=False) üretim kullanıldığı için temperature/top_p parametreleri dikkate alınmaz.
        - GPU belleğini korumak için giriş tensor’u .to(model.device) ile doğru cihaza aktarılır.
        - Dönen yanıt, </CEVAP> token’ı veya EOS görüldüğünde kesilir, 
          dolayısıyla ek string ayrıştırma gerektirmez.
    """

    # Eğer kullanıcı özel bir sonlandırıcı token id’si vermemişse
    # tokenizer’dan </CEVAP> etiketinin tamsayı id’sini bul.
    if eos_token_id is None:
        eos_token_id = tokenizer.convert_tokens_to_ids("</CEVAP>")

    # Girdi cümlesini token’layıp model’in çalıştığı cihaza (CPU/GPU) gönder.
    # return_tensors="pt" ⇒ PyTorch tensörü döndür.
    giris_ids = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Model.generate:
    #   - do_sample=False  ⇒ Greedy arama
    #   - max_new_tokens   ⇒ Çıkış uzunluk sınırı
    #   - no_repeat_ngram_size ⇒ Yinelenen n-gram’ları engelle
    #   - repetition_penalty   ⇒ Tekrarlayan token’lara ceza
    #   - eos_token_id         ⇒ </CEVAP> görüldüğünde dur
    #   - pad_token_id         ⇒ Eksik durumlar için padding (eos_token) kullan
    cikti_ids = model.generate(
        **giris_ids,
        max_new_tokens=max_new_tokens,
        do_sample=False,  # deterministik üretim
        eos_token_id=eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=no_repeat_ngram_size,
        repetition_penalty=repetition_penalty,
    )

    # generate çıktısında, ilk kısım girilen prompt’un id’leridir.
    # Biz sadece yeni üretilen token’ları decode etmek istiyoruz.
    yeni_tokenlar = cikti_ids[0][giris_ids["input_ids"].shape[-1]:]

    # Token’ları metne çevir. skip_special_tokens=True ⇒ <|endoftext|> vb. temizle
    yanit = tokenizer.decode(yeni_tokenlar, skip_special_tokens=True)

    # </CEVAP> etiketinden sonrası gereksiz; split ile kes ve trim’le.
    return yanit.split("</CEVAP>")[0].strip()

# Sonuç üretme kısmı

## Veri kümesi oku

In [ ]:
df_sorular = pd.read_csv(veri_kumesi_yolu)
print(f"{len(df_sorular)} soru yüklendi.")

## Her model için sonuç üret ve kaydet

In [ ]:
# sonuçlar dizinini oluştur
os.makedirs(sonuc_dizini, exist_ok=True)

In [ ]:
# --------------------------------------------------------------------------- #
#  Tüm modelleri tek tek dolaşarak, her birine karşılık sınama sorularının
#  cevaplarını üretip .csv dosyasına kaydeden ana döngü
# --------------------------------------------------------------------------- #

for model_etiket, model_folder in model_yollari.items():
    # ----------------------------------------------------------------------- #
    # 1) Çıktı dosyası yolu oluşturma
    # ----------------------------------------------------------------------- #
    # Örneğin `model_etiket = "gpt2_medium_gpt4o"` ise:
    #   cikti_yolu = "<kok_dizin>/sonuclar/gpt2_medium_gpt4o.csv"
    cikti_yolu = os.path.join(sonuc_dizini, f"{model_etiket}.csv")

    # Aynı model için çıktı dosyası daha önce oluşturulmuşsa hesaplama
    # tekrarlanmasın; zaman ve kaynak kazancı sağlanır.
    if os.path.exists(cikti_yolu):
        print(f"[SKIP] '{cikti_yolu}' zaten var, geçiliyor.")
        continue  # döngünün bir sonraki modeline atla

    # ----------------------------------------------------------------------- #
    # 2) Model + Tokenizer yükleme
    # ----------------------------------------------------------------------- #
    # model_yukle() fonksiyonu:
    #   • HuggingFace model klasörünü / hub adını `model_folder` argümanı ile alır
    #   • Uygun veri tipinde (dtype) ve uygun cihaza (CPU / GPU) yerleştirilmiş
    #     AutoModelForCausalLM nesnesi ve ilişkili AutoTokenizer döndürür
    model, tokenizer = model_yukle(model_folder)

    # `model.eval()`  ⇒  Eğitim modunu kapat, tahmin (inference) moduna geç
    #                  (dropout, layer norm gibi davranışlar sabitlenir).
    model.eval()

    # ----------------------------------------------------------------------- #
    # 3) Soru listesi üzerinde döngü – her bir sorudan cevap üret
    # ----------------------------------------------------------------------- #
    cevaplar = []  # Üretilen cevapları tutacak liste

    # tqdm → ilerleme çubuğu; iterasyon durumunu terminalde gösterir
    for soru in tqdm(df_sorular["soru"], desc=model_etiket):
        # Soruyu, modelin eğitildiği özel etiketli prompt biçimine çevir
        prompt = soruyu_girdi_bicimine_cevir(soru)

        # cevap_uret() → deterministik (greedy) generate çağrısı yapar
        cevap  = cevap_uret(model, tokenizer, prompt)

        # Listeye ekle
        cevaplar.append(cevap)

    # ----------------------------------------------------------------------- #
    # 4) Sonuç DataFrame’i oluştur ve diske kaydet
    # ----------------------------------------------------------------------- #
    #   • Başlangıçta okunan df_sorular DataFrame'inin aynısını kopyalayıp
    #     yanına yeni bir "cevap" sütunu ekliyoruz.
    df_sonuclar = df_sorular.copy()
    df_sonuclar["cevap"] = cevaplar

    # .csv olarak kaydet → Unicode karakterler için encoding="utf-8"
    df_sonuclar.to_csv(cikti_yolu, index=False, encoding="utf-8")
    print(f"[OK] {model_etiket} sonuçları '{cikti_yolu}' dosyasına kaydedildi.")

    # ----------------------------------------------------------------------- #
    # 5) Bellek temizliği
    # ----------------------------------------------------------------------- #
    # Bir sonraki model yüklenmeden önce GPU belleğini boşaltmak kritik.
    #   • `del model`           → Python referansını sil
    #   • `torch.cuda.empty_cache()` → PyTorch'un ayrılmış fakat kullanılmayan
    #                                   GPU belleğini boşalt
    del model
    torch.cuda.empty_cache()